# 01. Apache Iceberg 아키텍처 개요

이 노트북에서는 Apache Iceberg의 **3-Layer 아키텍처**를 실제 파일 구조를 통해 확인합니다.

## Iceberg 3-Layer 아키텍처

Apache Iceberg 테이블은 세 개의 계층으로 구성됩니다.

```
┌─────────────────────────────────┐
│         Catalog Layer           │
│  (테이블 이름 → 현재 metadata   │
│   파일 위치를 가리키는 포인터)    │
└──────────────┬──────────────────┘
               │
               ▼
┌─────────────────────────────────┐
│        Metadata Layer           │
│  metadata.json → manifest list  │
│  → manifest files               │
│  (스냅샷, 스키마, 파티션 정보,   │
│   파일 수준 통계 등)             │
└──────────────┬──────────────────┘
               │
               ▼
┌─────────────────────────────────┐
│          Data Layer             │
│  실제 데이터가 저장된            │
│  Parquet / ORC / Avro 파일들    │
└─────────────────────────────────┘
```

### 각 계층의 역할

| 계층 | 구성 요소 | 역할 |
|------|-----------|------|
| **Catalog Layer** | Hadoop FS / Hive Metastore / REST 등 | 테이블 이름을 현재 metadata.json 파일 경로로 매핑 |
| **Metadata Layer** | metadata.json, snap-*.avro, manifest-*.avro | 테이블 스냅샷, 스키마, 파티션 정보, 파일 수준 통계 관리 |
| **Data Layer** | .parquet 파일들 | 실제 레코드가 저장된 데이터 파일 |

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, count_files, total_size

In [ ]:
spark = create_spark_session("ArchOverview")

## 테이블 생성

`PARTITIONED BY (months(order_date))`를 사용하여 월 단위 파티션을 설정합니다.  
이것은 Iceberg의 **Hidden Partitioning** 기능으로, 사용자가 파티션 컬럼을 직접 관리할 필요가 없습니다.

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.lab")
spark.sql("DROP TABLE IF EXISTS demo.lab.arch_orders")

spark.sql("""
    CREATE TABLE demo.lab.arch_orders (
        order_id     BIGINT,
        customer_id  BIGINT,
        product_name STRING,
        order_date   DATE,
        amount       DOUBLE,
        status       STRING
    )
    USING iceberg
    PARTITIONED BY (months(order_date))
""")

print("테이블 생성 완료: demo.lab.arch_orders")

## 데이터 삽입

100건의 랜덤 주문 데이터를 생성하여 Iceberg 테이블에 삽입합니다.

In [ ]:
orders = generate_orders(num_records=100, seed=42)
df = to_spark_df(spark, orders)

df.writeTo("demo.lab.arch_orders").append()

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.arch_orders").collect()[0]["cnt"]
print(f"삽입 완료: {count}건")

## Warehouse 디렉토리 구조 탐색

`show_tree`로 Iceberg 테이블이 파일 시스템에 어떤 구조로 저장되는지 확인합니다.

In [ ]:
TABLE_PATH = "/home/jovyan/data/warehouse/lab/arch_orders"

show_tree(TABLE_PATH)

## 파일 통계

In [ ]:
parquet_count = count_files(TABLE_PATH, ext=".parquet")
json_count = count_files(TABLE_PATH, ext=".json")
avro_count = count_files(TABLE_PATH, ext=".avro")
total = total_size(TABLE_PATH)

print(f"Parquet 파일 (Data Layer):    {parquet_count}개")
print(f"JSON 파일 (Metadata Layer):   {json_count}개")
print(f"Avro 파일 (Metadata Layer):   {avro_count}개")
print(f"전체 크기:                    {total / 1024:.1f} KB")

## 관찰 포인트

디렉토리 구조를 보면 크게 두 영역으로 나뉩니다:

### Data Layer (`data/` 디렉토리)
- `data/` 안에 파티션 디렉토리들이 존재 (예: `order_date_month=2024-01/`)
- 각 파티션 안에 `.parquet` 파일 = **실제 데이터**
- Parquet는 **컬럼 기반(columnar)** 포맷으로, 분석 쿼리에 최적화

### Metadata Layer (`metadata/` 디렉토리)
- `metadata.json` = 테이블의 **현재 상태** (스키마, 파티션 스펙, 스냅샷 목록 등)
- `snap-*.avro` = **Manifest List** (어떤 manifest 파일이 이 스냅샷에 포함되는지)
- `*.avro` (snap- 접두어 없음) = **Manifest File** (어떤 data 파일이 포함되는지 + 통계)

### Catalog Layer
- Hadoop catalog의 경우, `version-hint.text` 파일이 현재 metadata.json의 버전을 가리킵니다
- 이것이 Catalog Layer의 "포인터" 역할을 합니다

---

**다음 노트북에서 이 파일들의 내부를 분석합니다.** Data Layer의 Parquet 파일 내부 구조와 Metadata Layer의 JSON/Avro 파일 내용을 직접 읽어보겠습니다.

In [ ]:
spark.stop()
print("Spark 세션 종료")